### Logistic回归分类器

#### 基于主要思想: 根据现有数据对分类边界线建立回归公式，以此进行分类
> 1. 优点：计算代价不高，以与理解和实现
> 2. 缺点：容易欠拟合，分类精度不高

#### 核心思想：
> 1. 首先确定分类器的基础函数，本例子为二类分类，所以采用在大尺度上可近似为Unit Step的Sigmoid函数   
Sigmoid: $f(z) = \frac{1}{1+e^{-z}}$
> 2. 回归: 对数据进行直线线性拟合的过程称为回归，即找到一条直线，使得数据点到直线的距离最小。在此例中，我们假设Sigmoid函数的输入为$z$，那么有:   
$z = \sum_{i=1}^{n}w_ix_i$，其中$w_i$为回归系数，$x_i$为输入数据，将$w_i$写作向量$\mathbf{w}$，则这个向量就是训练数据集的目标最佳回归系数向量。
> 3. 梯度上升法(最优化算法)   
假设函数$f(x,y)$，记其梯度为$\nabla f(x,y)$，该函数连续且处处可微，则其梯度为:   
$\nabla f(x,y) = \begin{pmatrix} \frac{\partial f}{\partial x} & \frac{\partial f}{\partial y} \end{pmatrix}^T$   
它表示要沿x轴方向移动$\frac{\partial f}{\partial x}$，沿y轴方向移动$\frac{\partial f}{\partial y}$
梯度上升算法到达每个点都会重新估计移动的方向，循环迭代直到满足停止条件，梯度算子总能保证选取的是最佳移动方向，而这个梯度算子在算法中只起到指引作用，还需要乘以步长值$\alpha$得到最终的移动量，那么梯度上升算法的迭代公式如下：   
$w:=w+\alpha\nabla_w f(w)$   
同理，对于梯度下降算法，其迭代公式为: $w:=w-\alpha\nabla_w f(w)$


In [ ]:
from numpy import *

def sigmoid(inX):
    """
    sigmoid函数
    """
    return 1.0 / (1 + exp(-inX))

def gradAscent(dataMatIn, classLabels):
    """
    梯度上升算法
    :param  dataMatIn   输入的数据矩阵
    :param  classLabels 类别标签
    :return 权重
    """
    # 将输入的数据矩阵转换为numpy的mat类型
    dataMatrix = matrix(dataMatIn)
    # 将类别标签转换为列向量，以方便矩阵运算
    labelMat = matrix(classLabels).transpose()
    # 获取输入矩阵的形状，m为样本数(行数)，n为特征数(列数)
    m, n = shape(dataMatrix)
    # 设置步长
    alpha = 0.001
    # 设置最大迭代次数
    maxCycles = 500
    # 初始化权重矩阵为1矩阵，形状为(特征数，1)
    weights = ones((n, 1))
    for k in range(maxCycles):
        # 计算当前权重下的预测值h，这里dataMatrix * weights返回的是一个维数为100的列向量，所以h也是一个列向量
        h = sigmoid(dataMatrix * weights)
        # 计算预测值与实际标签的误差，返回的是列表(行向量)
        error = (labelMat - h)
        # 更新权重列向量
        weights = weights + alpha * dataMatrix.transpose() * error
    # 返回最终的权重矩阵
    return weights

### 随机梯度上升
- 以此算法代替梯度上升求解最优回归系数向量的目的是为了减少计算复杂度
- 随机梯度上升算法的相对于梯度上升法的改进是：每次仅用一个样本点进行增量式更新回归系数，所以随机梯度上升算法实际上属于在线学习算法，与之对应的是批处理学习算法，随机梯度上升方法解决了模型训练时间过长的问题，随机梯度上升法在训练过程中可以逐步逼近最优解，判断一个优化算法优劣的可靠标准是看它是否会在最优解附近收敛，而随机梯度上升算法要进行到收敛的过程是需要时间的

In [ ]:
def stocGradAscent(dataMatrix, classLabels, numIter=150):
    # 将数据矩阵转换为numpy数组
    dataMatrix = array(dataMatrix)
    # 获得样本数量和特征数量
    m ,n = shape(dataMatrix)
    # 初始化权重向量为1
    weights = ones(n)
    # 开始迭代
    for j in range(numIter):
        # 获得样本索引列表
        dataIndex = list(range(m))
        for i in range(m):
            # 每次迭代时都会调整步长以缓解数据波动，随着迭代慢慢减小
            alpha = 4/(1.0 + j + i) + 0.01
            # 选取随机样本进行更新回归系数，这个方式可以减少数据的周期性波动
            randIndex = int(random.uniform(0, len(dataIndex)))
            # 计算误差
            h = sigmoid(sum(dataMatrix[randIndex] * weights))
            error = classLabels[randIndex] - h
            # 更新权重
            weights = weights + alpha * error * dataMatrix[randIndex]
            # 移除已经使用过的样本
            dataIndex.pop(randIndex)
    return weights
